In [2]:
import pandas as pd
import glob
from math import nan
dfd = pd.DataFrame.from_dict

Loading Data (permit data here)

In [ ]:
df = pd.read_csv('Permit Mining Results Sept2024.csv') # replace filepath or filename

In [ ]:
df.info()

In [ ]:
# prepares list of upgrade ages, WILL HAVE TO EDIT based on permit data

upgrade_types = list(df)[55:85]
upgrade_ages = [i + " age" for i in upgrade_types]

print(upgrade_ages)

The purpose of the first section is to prepare the dataframe that contains each property and its corresponding upgrades. Additional information is also pulled from the permit data (county, city, mailing address, and use code). The first cell will work with any set of permits, and the second transfers the permit data to the property dictionary.

In [6]:
# Function for calculating years since last sale date or age of property.
# The only difference is that sale date uses months

def get_years(date, month_flag=0):
    if len(date) < 4:
        return nan
    year = float(date[0:4])
    if month_flag == 1:
        month = date[4:6]
        year = year + (float(month) - 1)/12 # months round down
    return 2024 - year

In [7]:
property_upgrades = {}

# to easily switch between addresses and apns as keys if needed
key_type = 'APN'
first = 'PropertyAddressFull'

categories_1 = [
    first,
    'Permits',
    'City',
    'CountyName',
    'MailingAddress',
    'Use Code',
    'Last Sale Date',
    'Year Built',
    'Year Since Last Sale',
    'Property Age'
    ] 

categories = categories_1 + upgrade_types + upgrade_ages

# This builds up the dictionary of properties->upgrades, which is filled in afterwards
for index, row in df.iterrows():
    if key_type == 'APN':
        prop = str(row[key_type])
    else:
        prop = str(row[key_type]).lower()
    if prop not in property_upgrades.keys():
        property_upgrades[prop] = {i: 0 for i in categories}
        property_upgrades[prop][first] = row[first]
        property_upgrades[prop]['Permits'] = 1
        property_upgrades[prop]['City'] = row['PropertyAddressCity']
        property_upgrades[prop]['CountyName'] = row['CountyName']
        property_upgrades[prop]['MailingAddress'] = row['ContactOwnerMailAddressFull'] 
        property_upgrades[prop]['Use Code'] = row['Use Code']
        property_upgrades[prop]['Last Sale Date'] = row['Last Sale Date']
        property_upgrades[prop]['Year Built'] = row['Year Built']
        property_upgrades[prop]['Year Since Last Sale'] = get_years(str(row['Last Sale Date']), 1)
        property_upgrades[prop]['Property Age'] = get_years(str(row['Year Built']))
    else:
        property_upgrades[prop]['Permits'] += 1

In [ ]:
# to transfer the data

# Needed so this works correctly (note, only for old permit dataset)
# df = df.rename(columns={'Heat Pump Water Heater Flag': 'Heat Pump Water Heater', 'Air Source Heat Pump Flag': 'Air Source Heat Pump'})

# can be changed to include any already marked upgrade categories
old_u = upgrade_types

# pulls old upgrade data
for index, row in df.iterrows():
    if key_type == 'APN':
        prop = str(row[key_type])
    else:
        prop = str(row[key_type]).lower()
    for u in old_u:
        if row[u] == 1:
            property_upgrades[prop][u + ' age'] = row['EffectiveDate']
            property_upgrades[prop][u] = 1
    if property_upgrades[prop]['Electrical Panel Upgrade'] == 1:
        property_upgrades[prop]['Upgrade Amp'] = row['Upgrade Amp']

old_result = dfd(property_upgrades, orient='index')
old_result.head()

In [9]:
# saving data
# technically this can be skipped, added as a checkpoint

old_result.to_csv('propdata.csv') # set own filepath

This section takes upgrade age categories and updates the values to reflect the upgrade age as of 2024 (input data values are the dates of installation)

In [ ]:
df = pd.read_csv('propdata.csv')
df = df.set_index('Unnamed: 0')
df.info()

In [11]:
# select the specific columns that contain time data (might need to edit)
upgrade_ages = list(df)[40:70]

In [12]:
# This function returns the age of all upgrades present with a given property.
# Returns NaN under the following conditions:
#   If upgrade is not present
#   If upgrade date is not given (in this case the date is marked in PERMIT DATA as 1900-01-01, a placeholder)
def get_age(data, time, upgrades):
    for upgrade in upgrades:
        u = str(data[upgrade])
        if u != '0':
            u_year = u[0:4]
            if u_year == '1900':
                data[upgrade] = nan
            else:
                u_month = u[5:7]
                u_month_frac = (float(u_month) - 1)/12
                u_time = float(u_year) + u_month_frac
                age = time - u_time
                age = round(age, 3)
                data[upgrade] = str(age)
        else:
            data[upgrade] = nan

In [ ]:
# data 
data = df.to_dict(orient='index')

time = 2024

# update data
for property in data.keys():
    p_data = data[property]
    get_age(p_data, time, upgrade_ages)

new_df = dfd(data, orient='index')
new_df.head()

In [14]:
new_df.to_csv("property_upgrade_data_final.csv")

The purpose of the next section is to separate the properties based on their use code into Single Family (SF), Two to Four Family (twofour), Five+ Family (fiveplus), and commercial properties (comm). It does this with address OR APN, depending on user instruction. There is also a simple version which splits based on use code, because as of Sept 2024 property use codes have been added to the main permit data file, and are save above.

Loading data (property data here)

In [ ]:
data = pd.read_csv('property_upgrade_data_final.csv')
data.info()

In [ ]:
data = data.rename(columns={'Unnamed: 0': 'APN'})
data.head()

In [6]:
# Functions

# Function to connect all address parts into a single string for comparison since
# permit data contains full addresses but use code data does not. Takes row (property) as input,
# returns address (in lowercase) as a string. Not necessary now that use codes are in permit data,
# but kept for completeness
def addr_full(row):
    address = ''
    number = str(row['SITE_HOUSE_NUMBER'])
    direction = str(row['SITE_DIRECTION'])
    street = str(row['SITE_STREET_NAME'])
    mode = str(row['SITE_MODE'])
    if number != 'nan':
        index = number.find('.')
        if index != -1:
            number = number[:index]
        address += number
    if direction != 'nan':
        address = address + ' ' + direction
    if street != 'nan':
        address = address + ' ' + street
    if mode != 'nan':
        address = address + ' ' + mode
    return address.lower()

# Builds dictionaries for each property category. Needed because property use code data is separated
# by county. Takes pathname to use code data, dictionary to fill, and set of use codes.
def build_dict(pathname, prop_dict, use_codes, version):
    county_df = pd.read_csv(pathname)
    props = county_df[county_df['USE_CODE_STD_LPS'].isin(use_codes)]
    for index, row in props.iterrows():
        if version == 0:
            address = addr_full(row)
            prop_dict[address] = row
        elif version == 1:
            prop_dict[row['MASTER_PARCEL_APN']] = row

# Special function for separating 2-4 props from 5+ props within set of 3 overlapping use codes. Uses number
# of units to make decision. Essentially the same as build_dict but also takes user input for prop_type where type 1
# corresponds to 2-4 MF and type 2 corresponds to 5+ MF.
def fivefour(pathname, overlap, prop_dict, version, prop_type):
    county_df = pd.read_csv(pathname)
    props = county_df[county_df['USE_CODE_STD_LPS'].isin(overlap)]
    for index, row in props.iterrows():
        num_units = row['UNITS_NUMBER']
        if prop_type == 1:
            if num_units < 5:
                if version == 0:
                    address = addr_full(row)
                    prop_dict[address] = row
                elif version == 1:
                    prop_dict[row['MASTER_PARCEL_APN']] = row
        elif prop_type == 2:
            if num_units >= 5:
                if version == 0:
                    address = addr_full(row)
                    prop_dict[address] = row
                elif version == 1:
                    prop_dict[row['MASTER_PARCEL_APN']] = row

For the complicated version: first address->site data or apn->site data pairs are added to use code dictionaries, then in the following cell specific use code property upgrade data sets are extracted

For the simple version, the use code sets can be used directly (provided below), with the exception of the overlap set

In [ ]:
# this cell constructs the property type dictionaries with all corresponding property information

path = 'use_code_data/*.csv' # replace with own path, only works if all use code data is in one folder with no other .csv files

# use code sets
sf_use_codes = {1000, 1001, 1006, 1008, 1009, 1010, 1011, 1012, 1014, 1015, 1016, 1109}
twofour_use_codes = {1002, 1003, 1004, 1005, 1101, 1102, 1103, 1108}
five_use_codes = {1104, 1105, 1107, 1108, 1111, 1113, 9106, 9217}
commercial_use_codes = {2000, 2001, 2003, 2004, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2018, 2019, 2020,
                        2021, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2039, 2040, 2041, 
                        2042, 2043, 2044, 2045, 2046, 2047, 2048, 2050, 2051, 2052, 2054, 2058, 3000, 3001, 3002, 3003, 3004, 3005,
                        3006, 3007, 3008, 3010, 3011, 4000, 4001, 4002, 4003, 4004, 4006, 4007, 4008, 4009, 4010, 4011, 4012, 4013, 
                        4014, 4015, 4016, 4017, 4018, 4020, 4021, 4022, 4024, 4025, 4026, 4027, 4028, 4029, 4031, 4032, 5003, 6002,
                        9101, 9200, 9213, 9210, 9203, 9102, 9212, 9211, 9108, 9104, 9204, 9215, 9207, 9110, 9103, 9208, 9100, 9109,
                        9111, 9205, 9219, 9201, 9105, 9218, 9206, 9214}

# special use codes where 2-4 and 5+ MF overlap
overlap = {1100, 1110, 1112}

sf_dict = {}
twofour_dict = {}
five_dict = {}
comm_dict = {}

use_codes = [sf_use_codes, twofour_use_codes, five_use_codes, commercial_use_codes, overlap]
dicts = [sf_dict, twofour_dict, five_dict, comm_dict]

# note: 0 is for using addresses, 1 is for using APN
key_type = 1
for i in range(1, 3): # set right now to only build multifamily dictionaries because sf and comm aren't needed
    codes = use_codes[i]
    curr_dict = dicts[i]
    for pathname in glob.glob(path):
        build_dict(pathname, curr_dict, codes, key_type)
        if i in [1, 2]: # accounting for overlapping use codes
            fivefour(pathname, use_codes[4], curr_dict, key_type, i)

In [8]:
# property data for single family & commericial

sf_data = data[data['Use Code'].isin(sf_use_codes)]
comm_data = data[data['Use Code'].isin(commercial_use_codes)]

# property data for multifamily homes (needs dictionaries because of overlapping use codes)

twofour_data = data[data['APN'].isin(twofour_dict.keys())]
fiveplus_data = data[data['APN'].isin(five_dict.keys())]

In [ ]:
fiveplus_data.head()

In [13]:
sf_data.to_csv('SF_property_data.csv')
twofour_data.to_csv('MF2-4_property_data.csv')
fiveplus_data.to_csv('MF5plus_property_data.csv')
comm_data.to_csv('COMM_property_data.csv')

The next section is for upgrade timing, where upgrade information is updated with the time difference between either the year the property was built OR the year/month when the property was last purchased (chosen by user).

UPDATE: No longer used, now the upgrade ages have their own categories, ages are calculated in next section, left here for completeness

In [12]:
sf_data.to_csv('sfbin_fixed.csv')

In [ ]:
sf_data = pd.read_csv('sf_utime.csv')
# twofour_data = pd.read_csv('twofour_utime.csv')
# five_data = pd.read_csv('five_utime.csv')
# comm_data = pd.read_csv('comm_utime.csv')
sf_data.head()

In [ ]:
data = sf_data
data = data.rename(columns={'Unnamed: 0': 'Last Purchase Date'})
data = data.set_index('Address')
data.head()

In [ ]:
# adds a couple of columns that now necessary

data['Year Built'] = pd.Series(dtype=int)
data['Zip'] = pd.Series(dtype=str)
data.head()

In [41]:
# Functions

# Gets the difference in times (upgrade vs. either sale date or year built, version chosen with v where 1=sale date).
# Difference is returned as years with a decimal component as to be convenient for computation.
# Returns NaN under the following circumstances:
#   If earier date is not present
#   If upgrade date is earlier than "earlier" date (applies to sale date case)
#   If property last sale date is NaN (in the sale date case)
def get_time(u, o, v):

    if o:
        # process u
        u_year = u[0:4]
        u_month = u[5:7]
        u_month_frac = (float(u_month) - 1)/12
        u_time = float(u_year) + u_month_frac

        # process o
        if v == 1:
            if str(o) != 'n':
                o_year = o[0:4]
                o_month = u[4:6]
                o_month_frac = (float(o_month) - 1)/12
                o_time = float(o_year) + o_month_frac
            else:
                o_time = nan
        else:
            o_time = o

        # compare
        if u_time > o_time:
            time = u_time - o_time
        else:
            time = nan

        return time
    else:
        return nan

# goes through each upgrade and, if it is present, replaces upgrade date with time
def update_times(data, row, address, og_date, version):
    
    upgrades = [
        'Heat Pump Water Heater',
        'Air Source Heat Pump',
        'Solar PV',
        'Battery storage',
        'Electric Vehicle Charger',
        'Electrical Panel Upgrade',
        'Transformer',
        'Water Heater',
        'Cool Roof',
        'Kitchen Remodel',
        'Bathroom Remodel',
        'Other Remodel',
        'AC',
        'Reroof',
        'Spa/Pool',
        'Addition',
        'Lighting Install',
        'Wall Heater',
        'Tankless Water Heater'
    ]

    for upgrade in upgrades:
        upgrade_date = str(row[upgrade])
        if upgrade_date != '0':
            time = get_time(upgrade_date, og_date, version)
            if time:
                time = round(time, 3)
            data[address][upgrade] = time
        else: # sets all 0s to NaNs because while 0 is fine for the boolean upgrade data, it is a problem for computation
            data[address][upgrade] = nan

In [42]:
# goes through property upgrade data to update the data

f = data.to_dict(orient='index')
curr_dict = sf_dict.copy()

for address in f.keys():
    
    # getting last date of sale from use code data and adding to main frame
    pre_date = str(curr_dict[address]['LAST_SALE_DATE_TRANSFER'])
    pre_date = pre_date[:len(pre_date)-2]
    fin_date = pre_date[0:4] + '-' + pre_date[4:6] + '-' + pre_date[6:8]
    if fin_date == 'n--':
        f[address]['Last Purchase Date'] = nan
    else:
        f[address]['Last Purchase Date'] = fin_date

    # getting year built from use code data and adding to main frame
    yr_blt = curr_dict[address]['YR_BLT']
    if yr_blt:
        f[address]['Year Built'] = yr_blt
    else:
        f[address]['Year Built'] = nan
    
    # updating times for all upgrades, uncomment the desired version
    # update_times(f, f[address], address, yr_blt, 0)
    update_times(f, f[address], address, pre_date, 1)

    # adds zip code
    Zip = str(curr_dict[address]['SITE_ZIP'])
    f[address]['Zip'] = Zip[:len(Zip)-2]

In [ ]:
f_data_new = dfd(f, orient='index')
f_data_new.head()

In [44]:
f_data_new.to_csv('sf_u_tenure.csv')